**Разработка модели машинного обучения**
1. Обучающая выборка данных разделена на обучающую и тестовую выборку
2. Испытано как минимум три алгоритма разной сущности. Для оценки точности следует использовать
тестовую выборку ( RandomForest, GradientBoosting, LinearRegression)
3. Испытанные алгоритмы сравнены по точности, а также по скорости работы
4. Лучший алгоритм реализован в виде пайплайна, принимающего на вход пути к файлам, а на
выходе возвращает обученную модель, а также метрики модели на валидационной выборке в
виде словаря (необходимые метрики: средний коэффициент детерминации R2 и средний RMSE).

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import time
import joblib

In [6]:
def optimize_types(df):
    """Оптимизация типов данных в DataFrame."""
    for col in df.select_dtypes(include=['float']):
        df[col] = pd.to_numeric(df[col], downcast='float')
    for col in df.select_dtypes(include=['int']):
        df[col] = pd.to_numeric(df[col], downcast='integer')
    for col in df.select_dtypes(include=['object']):
        df[col] = df[col].astype('category')
    return df

In [7]:
def load_data_in_chunks(X_path, y_path, chunk_size=100000):
    """Генератор для загрузки данных по частям."""
    for X_chunk, y_chunk in zip(pd.read_csv(X_path, chunksize=chunk_size), pd.read_csv(y_path, chunksize=chunk_size)):
        X_chunk = optimize_types(X_chunk)  # Оптимизация типов данных
        y_chunk = optimize_types(y_chunk)  # Оптимизация типов данных
        yield X_chunk, y_chunk


In [8]:
def load_full_data(X_path, y_path):
    """Загрузка всех данных из файлов по частям и объединение в один DataFrame."""
    X_data = pd.DataFrame()
    y_data = pd.DataFrame()

    for X_chunk, y_chunk in load_data_in_chunks(X_path, y_path):
        X_data = pd.concat([X_data, X_chunk], ignore_index=True)
        y_data = pd.concat([y_data, y_chunk], ignore_index=True)

    return X_data, y_data

Обучим 3 модели : случайный лес, градиентный бустинг и линейная регрессия

In [5]:
def train_and_evaluate(X, y):
    """Обучение и оценка моделей."""
    models = {
        'RandomForest': RandomForestRegressor(),
        'GradientBoosting': GradientBoostingRegressor(),
        'LinearRegression': LinearRegression()
    }
    
    results = {}
    
    # Разделение данных на обучающую и тестовую выборки
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    for name, model in models.items():
        start_time = time.time()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        end_time = time.time()
        
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        r2 = r2_score(y_test, y_pred)
        
        results[name] = {
            'RMSE': rmse,
            'R2': r2,
            'Time': end_time - start_time
        }
    
    return results

In [ ]:
def best_model_pipeline(X_path, y_path):
    """Основная функция для обработки данных и обучения моделей."""
    # Загрузка всех данных
    X_data, y_data = load_full_data(X_path, y_path)

    # Обучение и оценка моделей
    metrics = train_and_evaluate(X_data, y_data)

    # Сравнение результатов
    best_model_name = min(metrics, key=lambda k: metrics[k]['RMSE'])
    best_model = models[best_model_name]

    # Обучение лучшей модели на всей выборке
    best_model.fit(X_data, y_data)

    # Сохранение модели
    joblib.dump(best_model, 'best_model.pkl')

    return best_model, metrics

In [ ]:
X_path = '/home/c5/Рабочий стол/Модуль 4/BIG DATA/df_0.csv'  # Путь к файлу с признаками
y_path = '/home/c5/Рабочий стол/Модуль 4/BIG DATA/target_0.csv'  # Путь к файлу с целевой переменной

best_model, metrics = best_model_pipeline(X_path, y_path)

print("Лучшие метрики модели:")
print(metrics)